## Stage 1: Installing dependencies and environment setup


In [1]:
!pip install tensorflow-gpu==2.9.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 511.8 MB 25 kB/s 


In [2]:
!pip install pandas-datareader

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
! pip install pycryptodome pycryptodomex
! pip uninstall --yes pandas-datareader
! pip install git+https://github.com/raphi6/pandas-datareader.git@ea66d6b981554f9d0262038aef2106dda7138316

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.3 MB 3.9 MB/s 
     |████████████████████████████████| 2.3 MB 28.9 MB/s 
Found existing installation: pandas-datareader 0.9.0
Uninstalling pandas-datareader-0.9.0:
  Successfully uninstalled pandas-datareader-0.9.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/raphi6/pandas-datareader.git (to revision ea66d6b981554f9d0262038aef2106dda7138316) to /tmp/pip-req-build-sqznaslr
  Running command git clone -q https://github.com/raphi6/pandas-datareader.git /tmp/pip-req-build-sqznaslr
  Running command git rev-parse -q --verify 'sha^ea66d6b981554f9d0262038aef2106dda7138316'
  Running command git fetch -q https://github.com/raphi6/pandas-datareader.git ea66d6b981554f9d0262038aef2106dda7138316
  Running command git checkout -q ea66d6b981554f9d0262038aef2106dda7138316
  Create

## Stage 2: Importing project dependencies

In [4]:
import math
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas_datareader as data_reader

from tqdm import tqdm_notebook, tqdm
from collections import deque

In [5]:
tf.__version__

'2.9.2'

## Stage 3: Building the AI Trader network

In [6]:
from threading import active_count
class AI_Trader():

  # action_space = Stay, Buy, Sell
  def __init__(self, state_size, action_space=3, model_name='AITrader'):
    self.state_size = state_size
    self.action_space = action_space
    self.memory = deque(maxlen=2000)
    self.inventory = [] # list of all bought stocks.
    self.model_name = model_name

    self.gamma = 0.95
    self.epsilon = 1.0
    self.epsilon_final = 0.01
    self.epsilon_decay = 0.995

    self.model = self.model_builder()

  def model_builder(self):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(units=32, activation='relu', input_dim=self.state_size))
    model.add(tf.keras.layers.Dense(units=64, activation='relu'))
    model.add(tf.keras.layers.Dense(units=128, activation='relu'))
    # Why output layer's activation is 'linear'? : to modify actions in a continuous number.
    model.add(tf.keras.layers.Dense(units=self.action_space, activation='linear'))
    model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(lr=0.001))
    return model

  def trade(self, state):
    if random.random() <= self.epsilon:
      return random.randrange(self.action_space)

    actions = self.model.predict(state, verbose=0)
    return np.argmax(actions[0])

  def batch_train(self, batch_size):
    batch = []
    for i in range(len(self.memory) - batch_size + 1, len(self.memory)):
      batch.append(self.memory[i])

    for state, action, reward, next_state, done in batch:
      reward = reward
      if not done:
        reward = reward + self.gamma * np.amax(self.model.predict(next_state, verbose=0)[0])

      target = self.model.predict(state, verbose=0)
      target[0][action] = reward

      self.model.fit(state, target, epochs=1, verbose=0)
      
    if self.epsilon > self.epsilon_final:
      self.epsilon *= self.epsilon_decay

## Stage 4: Dataset preprocessing

### Defining helper functions

#### Sigmoid

In [7]:
def sigmoid(x):
  return 1 / (1 + math.exp(-x))

#### Price format function

In [8]:
def stocks_price_format(n):
  if n < 0:
    return '- $ {0:2f}'.format(abs(n))
  else:
    return '$ {0:2f}'.format(abs(n))

#### Dataset loader

In [9]:
def dataset_loader(stock_name):
  dataset = data_reader.DataReader(stock_name, data_source='yahoo')

  start_date = str(dataset.index[0]).split()[0]
  end_date = str(dataset.index[-1]).split()[0]
  close = dataset['Close']
  
  return close

### State creator

In [10]:
# timestep : the day in the data set that we want to predict for
# window_szie = how many previous days we want to consider in our state to predict the current one.
def state_creator(data, timestep, window_size):
  starting_id = timestep - window_size + 1
  
  if starting_id >= 0:
    windowed_data = data[starting_id:timestep+1]
  else:
    # -starting_id * [data[0]] : replicate data[0] as much as starting_id to the list.
    # why replicate data[0]? : to have the state with the wanted size.
    windowed_data = -starting_id * [data[0]] + list(data[0:timestep+1])

  state = []
  
  for i in range(window_size - 1):
    state.append(sigmoid(windowed_data[i+1] - windowed_data[i]))

  return np.array([state])

### Loading a dataset

In [11]:
stock_name = 'AAPL'
data = dataset_loader(stock_name)

In [12]:
data

Date
2018-01-02     43.064999
2018-01-03     43.057499
2018-01-04     43.257500
2018-01-05     43.750000
2018-01-08     43.587502
                 ...    
2022-12-21    135.449997
2022-12-22    132.229996
2022-12-23    131.860001
2022-12-27    130.029999
2022-12-28    126.040001
Name: Close, Length: 1257, dtype: float64

## Stage 5: Training the AI Trader

### Setting hyper parameters

In [13]:
window_size = 10
episodes = 0 # = epochs
batch_size = 32
data_samples = len(data) - 1

### Defining the Trader model

In [14]:
trader = AI_Trader(window_size)

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [15]:
trader.model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                352       
                                                                 
 dense_1 (Dense)             (None, 64)                2112      
                                                                 
 dense_2 (Dense)             (None, 128)               8320      
                                                                 
 dense_3 (Dense)             (None, 3)                 387       
                                                                 
Total params: 11,171
Trainable params: 11,171
Non-trainable params: 0
_________________________________________________________________


### Training loop

In [16]:
for episode in range(episodes + 1):
  print('Episode: {}/{}'.format(episode, episodes))
  state = state_creator(data, 0, window_size+1)
  total_profit = 0
  trader.inventory = []

  for t in tqdm(range(data_samples)):
    action = trader.trade(state)
    next_state = state_creator(data, t+1, window_size+1)
    reward = 0

    if action == 1: # Buying
      trader.inventory.append(data[t])
      print('AI Trader bought: ', stocks_price_format(data[t]))
    elif action == 2 and len(trader.inventory) > 0: #Selling
      buy_price = trader.inventory.pop(0)
      reward = max(data[t] - buy_price, 0)
      total_profit += data[t] - buy_price
      print('AI Trader sold: {} Profit: {}'.format(stocks_price_format(data[t]), stocks_price_format(data[t] - buy_price)))

    if t == data_samples - 1:
      done = True
    else:
      done = False

    trader.memory.append((state, action, reward, next_state, done))

    state = next_state

    if done:
      print('TOTAL PROFIT: {}'.format(total_profit))

    if len(trader.memory) > batch_size:
      trader.batch_train(batch_size)

  if episode % 10 == 0:
    trader.model.save('ai_trader_{}.h5'.format(episode))

Episode: 0/0


  0%|          | 0/1256 [00:00<?, ?it/s]

AI Trader bought:  $ 43.750000
AI Trader bought:  $ 43.587502
AI Trader bought:  $ 43.582500
AI Trader sold: $ 43.572498 Profit: - $ 0.177502
AI Trader sold: $ 43.820000 Profit: $ 0.232498
AI Trader bought:  $ 44.272499
AI Trader bought:  $ 44.047501
AI Trader sold: $ 44.775002 Profit: $ 1.192501
AI Trader bought:  $ 44.615002
AI Trader sold: $ 44.250000 Profit: - $ 0.022499
AI Trader sold: $ 44.259998 Profit: $ 0.212498
AI Trader sold: $ 42.877499 Profit: - $ 1.737503
AI Trader bought:  $ 41.945000
AI Trader sold: $ 40.125000 Profit: - $ 1.820000
AI Trader bought:  $ 38.787498
AI Trader bought:  $ 39.102501
AI Trader sold: $ 40.677502 Profit: $ 1.890003
AI Trader sold: $ 41.084999 Profit: $ 1.982498


  3%|▎         | 33/1256 [00:06<03:55,  5.19it/s]

AI Trader bought:  $ 42.962502


...

 99%|█████████▉| 1246/1256 [1:08:42<00:33,  3.31s/it]

AI Trader sold: $ 145.470001 Profit: $ 2.040009


100%|█████████▉| 1255/1256 [1:09:12<00:03,  3.34s/it]

TOTAL PROFIT: 25.08746337890625


100%|██████████| 1256/1256 [1:09:15<00:00,  3.31s/it]
